In [45]:
from initial_pop import *
from code_classes import *
import numpy as np

In [46]:
class Player:
    def __init__(self, name, position, skill, cost):
        self.name = name
        self.position = position
        self.skill = skill
        self.cost = cost

    def __str__(self):
        return f"{self.name} ({self.position}) - Skill: {self.skill}, Cost: {self.cost}M"


In [47]:
class Team:
    def __init__(self, players):
        self.players = players  # players is a list of Player objects
        self.validate_team()

    def validate_team(self):
        positions = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
        for player in self.players:
            if player.position not in positions:
                raise ValueError(f"Invalid player position: {player.position}")
            positions[player.position] += 1

        # Check the required structure
        if positions["GK"] != 1 or positions["DEF"] != 2 or positions["MID"] != 2 or positions["FWD"] != 2:
            raise ValueError("Each team must have 1 GK, 2 DEF, 2 MID, and 2 FWD.")

        # Check if the team exceeds salary cap
        total_salary = sum(player.cost for player in self.players)
        if total_salary > 750:
            raise ValueError(f"Team salary exceeds the cap: {total_salary}M")
    
    def get_total_salary(self):
        return sum(player.cost for player in self.players)

    def get_average_skill(self):
        return sum(player.skill for player in self.players) / len(self.players)

    def __str__(self):
        return "\n".join([str(player) for player in self.players])

In [48]:
class League:
    def __init__(self, teams):
        self.teams = teams
        self.validate_league()
                
    def validate_league(self):
        if len(self.teams) != 5:
            raise ValueError("The league must have exactly 5 teams.")

        player_names = set()
        for team in self.teams:
            # Explicitly call team.validate_team()
            team.validate_team()

            for player in team.players:
                if player.name in player_names:
                    raise ValueError(f"Player {player.name} is already in another team.")
                player_names.add(player.name)


    def is_valid(self):
        try:
            self.validate_league()
            return True
        except ValueError:
            return False

    def get_skill_std_dev(self):
        # Calculate average skill for each team
        avg_skills = [team.get_average_skill() for team in self.teams]
        # Return the standard deviation of the average skill levels of the teams
        return np.std(avg_skills)

    def __str__(self):
        return "\n\n".join([str(team) for team in self.teams])

In [68]:
def calculate_fitness(league):
    """
    Calculate fitness for a league (lower is better)

    1. If league is valid (formations, budget, unique players)
    2. Standard deviation of average skills (our main objective)
    """
    try:
        league.validate_league()
    except ValueError as e:
        print(f"Fitness validation failed: {e}")
        return 9999

    # If it's valid, compute and return the standard deviation of average skills
    return league.get_skill_std_dev()

In [50]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

In [51]:
# import random

# def create_valid_team_from_pool(player_pool):
#     max_attempts = 10
#     for _ in range(max_attempts):
#         gks = [p for p in player_pool if p.position == "GK"]
#         defs = [p for p in player_pool if p.position == "DEF"]
#         mids = [p for p in player_pool if p.position == "MID"]
#         fwds = [p for p in player_pool if p.position == "FWD"]

#         if len(gks) < 1 or len(defs) < 2 or len(mids) < 2 or len(fwds) < 2:
#             raise ValueError("Not enough players in the pool to form a valid team.")

#         selected_players = random.sample(gks, 1) + \
#                            random.sample(defs, 2) + \
#                            random.sample(mids, 2) + \
#                            random.sample(fwds, 2)

#         try:
#             team = Team(selected_players)
#             return team
#         except ValueError:
#             continue

#     raise ValueError("Failed to create a valid team after many attempts.")

# def create_valid_league(all_players, num_teams=5):
#     max_attempts = 10
#     for _ in range(max_attempts):
#         random.shuffle(all_players)
#         available_players = all_players.copy()
#         used_names = set()
#         teams = []

#         try:
#             for _ in range(num_teams):
#                 pool = [p for p in available_players if p.name not in used_names]
#                 team = create_valid_team_from_pool(pool)
#                 teams.append(team)
#                 used_names.update(p.name for p in team.players)
#             return League(teams)
#         except ValueError:
#             #print the error
#             print(f"Error creating league: {'e'}")
#             continue

#     raise ValueError("Failed to create a valid league after many attempts.")

# def generate_population(players, num_leagues=5):
#     population = []
#     for _ in range(num_leagues):
#         league = create_valid_league(players)
#         population.append(league)
#     return population


In [52]:
def create_valid_team_from_pool(player_pool):
    max_attempts = 100
    for _ in range(max_attempts):
        gks = [p for p in player_pool if p.position == "GK"]
        defs = [p for p in player_pool if p.position == "DEF"]
        mids = [p for p in player_pool if p.position == "MID"]
        fwds = [p for p in player_pool if p.position == "FWD"]

        if len(gks) < 1 or len(defs) < 2 or len(mids) < 2 or len(fwds) < 2:
            raise ValueError("Not enough players in the pool to form a valid team.")

        selected_players = random.sample(gks, 1) + \
                           random.sample(defs, 2) + \
                           random.sample(mids, 2) + \
                           random.sample(fwds, 2)

        try:
            team = Team(selected_players)
            return team
        except ValueError:
            continue

    raise ValueError("Failed to create a valid team after many attempts.")

def create_valid_league(all_players, num_teams=5):
    max_attempts = 100
    for _ in range(max_attempts):
        random.shuffle(all_players)
        available_players = all_players.copy()
        used_names = set()
        teams = []

        try:
            for _ in range(num_teams):
                pool = [p for p in available_players if p.name not in used_names]
                team = create_valid_team_from_pool(pool)
                teams.append(team)
                used_names.update(p.name for p in team.players)
            return League(teams)
        except ValueError:
            continue

    raise ValueError("Failed to create a valid league after many attempts.")

def generate_population(players, num_leagues=5):
    population = []
    for _ in range(num_leagues):
        league = create_valid_league(players)
        population.append(league)
    return population

In [53]:
population = generate_population(players, num_leagues=50)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_skill_std_dev():.2f}")


--- League 1 ---
Chris Thompson (GK) Skill: 80 Salary: €80.0M
Owen Parker (DEF) Skill: 88 Salary: €100.0M
Ethan Howard (DEF) Skill: 80 Salary: €70.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Austin Torres (MID) Skill: 82 Salary: €80.0M
Sebastian Perry (FWD) Skill: 95 Salary: €150.0M
Xavier Bryant (FWD) Skill: 90 Salary: €120.0M

Alex Carter (GK) Skill: 85 Salary: €90.0M
Caleb Fisher (DEF) Skill: 84 Salary: €85.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Spencer Ward (MID) Skill: 84 Salary: €85.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M
Elijah Sanders (FWD) Skill: 93 Salary: €140.0M

Ryan Mitchell (GK) Skill: 83 Salary: €85.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Julian Scott (FWD) Skill: 92 Salary: €130.0M
Zachary Nelson (FWD) Skill: 86 Salary: €92.0M

Blake Henderson (GK) Skill: 87 S

In [54]:
import numpy as np

## lol this is a mutator :()

In [55]:
# def mutate_within_league_by_position(league):
#     positions = ['GK', 'DEF', 'MID', 'FWD']

#     # Perform mutation to generate a mutated league
#     mutated_league = deepcopy(league)

#     team1, team2 = random.sample(mutated_league.teams, 2)
#     position = random.choice(positions)

#     players_team1 = [p for p in team1.players if p.position == position]
#     players_team2 = [p for p in team2.players if p.position == position]

#     if not players_team1 or not players_team2:
#         return None  # No valid players to swap

#     p1 = random.choice(players_team1)
#     p2 = random.choice(players_team2)

#     # Perform swap virtually to check
#     temp_team1_players = [p for p in team1.players if p != p1] + [p2]
#     temp_team2_players = [p for p in team2.players if p != p2] + [p1]

#     try:
#         # Try creating new Team objects to check for salary cap and full validation
#         new_team1 = Team(temp_team1_players)
#         new_team2 = Team(temp_team2_players)

#         # If both teams are valid, apply changes
#         team1.players = new_team1.players
#         team2.players = new_team2.players

#         # Successful mutation print message
#         print(f"Successful mutation swapping {p1.name} ↔ {p2.name} in position {position}")

#         return mutated_league  # Valid mutated league created

#     except ValueError as e:
#         print(f"Invalid mutation")
#         return None  # Invalid mutation, return None


In [57]:
# parent_league = random.choice(population)

# child_league = mutate_within_league_by_position(parent_league)

In [58]:
# # Print parents' constitution
# print("Parent League:")
# for i, team in enumerate(parent_league.teams):
#     print(f"\nTeam {i+1}:")
#     print(team)
#     print(f"Average Skill: {team.get_average_skill():.2f}")
# print(f"Fitness: {calculate_fitness(parent_league):.2f}")

# # Print child league constitution
# print("\nChild League:")
# if child_league:
#     for i, team in enumerate(child_league.teams):
#         print(f"\nTeam {i+1}:")
#         print(team)
#         print(f"Average Skill: {team.get_average_skill():.2f}")
#     print(f"Fitness: {calculate_fitness(child_league):.2f}")
# else:
#     print(f"Fitness: {calculate_fitness(child_league):.2f}")

## NOW CROSSOVER

# Crossover between teams within a league

In [134]:
from copy import deepcopy
import random

def crossover_swap_whole_position(league1, league2):
    """
    Perform a crossover between two leagues by swapping all players of a randomly selected position (GK, DEF, MID, or FWD)
    between corresponding teams in each league.

    This crossover selects one position and swaps players in that position between matching teams (i.e., Team 1 of League 1 
    with Team 1 of League 2, and so on). If a team does not have any players in the selected position, the swap is skipped 
    for that specific pair of teams.

    Example:
    --------
    Consider two parent leagues, Parent 1 and Parent 2, each containing 3 teams:

    Parent 1:
    - Team 1: GK(A1), DEF(A2, A3), MID(A4, A5), FWD(A6, A7)
    - Team 2: GK(B1), DEF(B2, B3), MID(B4, B5), FWD(B6, B7)
    - Team 3: GK(C1), DEF(C2, C3), MID(C4, C5), FWD(C6, C7)

    Parent 2:
    - Team 1: GK(D1), DEF(D2, D3), MID(D4, D5), FWD(D6, D7)
    - Team 2: GK(E1), DEF(E2, E3), MID(E4, E5), FWD(E6, E7)
    - Team 3: GK(F1), DEF(F2, F3), MID(F4, F5), FWD(F6, F7)

    Step 1: Randomly select a position (e.g., "DEF")

    Step 2: For each pair of teams at the same index, swap their players at that position

    Step 3: After the swap, the child leagues will look like this (assuming position "DEF" was chosen):

    Child 1:
    - Team 1: GK(A1), DEF(D2, D3), MID(A4, A5), FWD(A6, A7)
    - Team 2: GK(B1), DEF(E2, E3), MID(B4, B5), FWD(B6, B7)
    - Team 3: GK(C1), DEF(F2, F3), MID(C4, C5), FWD(C6, C7)
    
    Child 2:
    - Team 1: GK(D1), DEF(A2, A3), MID(D4, D5), FWD(D6, D7)
    - Team 2: GK(E1), DEF(B2, B3), MID(E4, E5), FWD(E6, E7)
    - Team 3: GK(F1), DEF(C2, C3), MID(F4, F5), FWD(F6, F7)

    Parameters:
    -----------
    league1 : League
        The first parent league to perform crossover from.

    league2 : League
        The second parent league to perform crossover from.

    Returns:
    --------
    tuple : (League, League)
        A tuple containing the two child leagues after the crossover. Each child league contains a mix of players at the 
        selected position swapped between corresponding teams.
    """
    # Deep copy leagues to avoid altering the originals
    child1 = deepcopy(league1)
    child2 = deepcopy(league2)

    positions = ['GK', 'DEF', 'MID', 'FWD']
    chosen_pos = random.choice(positions)
    print(f"Swapping all players at position: {chosen_pos}")

    for t1, t2 in zip(child1.teams, child2.teams):
        # Get players by position from both teams
        p1 = [p for p in t1.players if p.position == chosen_pos]
        p2 = [p for p in t2.players if p.position == chosen_pos]

        # Skip if either team lacks players at the selected position
        if not p1 or not p2:
            print(f"One of the teams lacks players at {chosen_pos}. Skipping swap for this pair.")
            continue

        # Make sure the numbers match, swap only as many players as available in both teams
        min_len = min(len(p1), len(p2))

        # Swap equal number of players for the selected position
        for i in range(min_len):
            player1 = p1[i]
            player2 = p2[i]

            # Replace by identity to avoid name conflicts
            def replace_player(team, old_p, new_p):
                team.players = [new_p if p is old_p else p for p in team.players]

            replace_player(t1, player1, player2)
            replace_player(t2, player2, player1)

    return child1, child2


In [139]:
# Select two different parent leagues
parent_league1 = random.choice(population)
parent_league2 = random.choice(population)
while parent_league1 == parent_league2:
    parent_league2 = random.choice(population)

# Show parent fitness
print("\n--- Parent League 1 ---")
for i, team in enumerate(parent_league1.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(parent_league1):.2f}")

print("\n--- Parent League 2 ---")
for i, team in enumerate(parent_league2.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(parent_league2):.2f}")

# Apply position-based crossover
child1, child2 = crossover_swap_whole_position(parent_league1, parent_league2)

# Show child fitness
print("\n--- Child League 1 ---")
for i, team in enumerate(child1.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child1):.2f}")  # <-- this is fixed

print("\n--- Child League 2 ---")
for i, team in enumerate(child2.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child2):.2f}")


--- Parent League 1 ---
Team 1 - Avg Skill: 86.57 | Budget: 687.00
Team 2 - Avg Skill: 86.86 | Budget: 695.00
Team 3 - Avg Skill: 85.86 | Budget: 667.00
Team 4 - Avg Skill: 87.57 | Budget: 725.00
Team 5 - Avg Skill: 85.14 | Budget: 650.00
Fitness: 0.83

--- Parent League 2 ---
Team 1 - Avg Skill: 88.43 | Budget: 750.00
Team 2 - Avg Skill: 83.71 | Budget: 594.00
Team 3 - Avg Skill: 84.71 | Budget: 625.00
Team 4 - Avg Skill: 87.71 | Budget: 725.00
Team 5 - Avg Skill: 87.43 | Budget: 730.00
Fitness: 1.84
Swapping all players at position: GK

--- Child League 1 ---
Team 1 - Avg Skill: 86.14 | Budget: 677.00
Team 2 - Avg Skill: 86.57 | Budget: 690.00
Team 3 - Avg Skill: 86.57 | Budget: 682.00
Team 4 - Avg Skill: 86.57 | Budget: 710.00
Team 5 - Avg Skill: 86.14 | Budget: 665.00
Fitness: 0.21

--- Child League 2 ---
Team 1 - Avg Skill: 88.86 | Budget: 760.00
Team 2 - Avg Skill: 84.00 | Budget: 599.00
Team 3 - Avg Skill: 84.00 | Budget: 610.00
Team 4 - Avg Skill: 88.71 | Budget: 740.00
Team 5

In [140]:
def print_league_details(league, league_name="League"):
    print(f"\n--- {league_name} Composition ---")
    for i, team in enumerate(league.teams):
        print(f"\nTeam {i+1}")
        for player in team.players:
            print(f"  {player.name} | Position: {player.position} | Skill: {player.skill} | Salary: {player.cost}")
        print(f"  → Avg Skill: {team.get_average_skill():.2f} | Total Salary: {team.get_total_salary():.2f}")

# Example: print parent_league1 in detail
print("\n--- Parent League 1 Details ---")
print_league_details(parent_league1, "Parent League 1")

# Example: print parent_league2 in detail
print("\n--- Parent League 2 Details ---")
print_league_details(parent_league2, "Parent League 2")

# Example: print child1 in detail

print("\n--- Child League 1 Details ---")
print_league_details(child1, "Child League 1")

# Example: print child2 in detail
print("\n--- Child League 2 Details ---")
print_league_details(child2, "Child League 2")


# Optional: print parent leagues or child2 as well
# print_league_details(parent_league1, "Parent League 1")
# print_league_details(child2, "Child League 2")



--- Parent League 1 Details ---

--- Parent League 1 Composition ---

Team 1
  Jordan Smith | Position: GK | Skill: 88 | Salary: 100.0
  Brayden Hughes | Position: DEF | Skill: 87 | Salary: 100.0
  Owen Parker | Position: DEF | Skill: 88 | Salary: 100.0
  Hunter Cooper | Position: MID | Skill: 83 | Salary: 85.0
  Austin Torres | Position: MID | Skill: 82 | Salary: 80.0
  Julian Scott | Position: FWD | Skill: 92 | Salary: 130.0
  Zachary Nelson | Position: FWD | Skill: 86 | Salary: 92.0
  → Avg Skill: 86.57 | Total Salary: 687.00

Team 2
  Alex Carter | Position: GK | Skill: 85 | Salary: 90.0
  Daniel Foster | Position: DEF | Skill: 90 | Salary: 110.0
  Caleb Fisher | Position: DEF | Skill: 84 | Salary: 85.0
  Gavin Richardson | Position: MID | Skill: 87 | Salary: 95.0
  Spencer Ward | Position: MID | Skill: 84 | Salary: 85.0
  Elijah Sanders | Position: FWD | Skill: 93 | Salary: 140.0
  Adrian Collins | Position: FWD | Skill: 85 | Salary: 90.0
  → Avg Skill: 86.86 | Total Salary: 695.

In [141]:
# Show child fitness and budgets
print("\n--- Child League 1 ---")
for i, team in enumerate(child1.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child1):.2f}")

print("\n--- Child League 2 ---")
for i, team in enumerate(child2.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child2):.2f}")


--- Child League 1 ---
Team 1 - Avg Skill: 86.14 | Budget: 677.00
Team 2 - Avg Skill: 86.57 | Budget: 690.00
Team 3 - Avg Skill: 86.57 | Budget: 682.00
Team 4 - Avg Skill: 86.57 | Budget: 710.00
Team 5 - Avg Skill: 86.14 | Budget: 665.00
Fitness: 0.21

--- Child League 2 ---
Team 1 - Avg Skill: 88.86 | Budget: 760.00
Team 2 - Avg Skill: 84.00 | Budget: 599.00
Team 3 - Avg Skill: 84.00 | Budget: 610.00
Team 4 - Avg Skill: 88.71 | Budget: 740.00
Team 5 - Avg Skill: 86.43 | Budget: 715.00
Fitness validation failed: Team salary exceeds the cap: 760.0M
Fitness: 9999.00


# Crossover between two leagues

In [698]:
from copy import deepcopy

def preset_team_mix_crossover(parent1, parent2):
    """
    Performs a deterministic crossover using a fixed team selection pattern from each parent.
    Ensures that no player appears more than once in the resulting child leagues.

    Pattern:
    Child 1: Teams 1, 3, 5 from parent1; Teams 2, 4 from parent2
    Child 2: Teams 1, 3, 5 from parent2; Teams 2, 4 from parent1

    Parameters:
    -----------
    parent1 : League
        First parent league.
    parent2 : League
        Second parent league.

    Returns:
    --------
    tuple : (League, League)
        Two new child leagues built from the preset team pattern. If duplicates are found,
        it raises an error.
    """

    def build_child(pattern, source1, source2):
        teams = []
        for idx in pattern:
            if idx[0] == 1:
                teams.append(deepcopy(source1.teams[idx[1]]))
            else:
                teams.append(deepcopy(source2.teams[idx[1]]))
        return League(teams)

    # Define the selection pattern: (source_league, team_index)
    pattern_child1 = [(1, 0), (2, 1), (1, 2), (2, 3), (1, 4)]
    pattern_child2 = [(2, 0), (1, 1), (2, 2), (1, 3), (2, 4)]

    child1 = build_child(pattern_child1, parent1, parent2)
    child2 = build_child(pattern_child2, parent1, parent2)

    # Validate for duplicate players
    def validate_unique_players(league, child_name):
        player_names = set()
        for i, team in enumerate(league.teams):
            for player in team.players:
                if player.name in player_names:
                    raise ValueError(
                        f"{child_name} is invalid: Duplicate player {player.name} in team {i+1}."
                    )
                player_names.add(player.name)

    try:
        validate_unique_players(child1, "Child 1")
        validate_unique_players(child2, "Child 2")
    except ValueError as e:
        print("Validation failed:", e)
        return None, None

    print("Preset team mix crossover successful.")
    return child1, child2


In [832]:
# Select two different parent leagues
parent_league1 = random.choice(population)
parent_league2 = random.choice(population)
while parent_league1 == parent_league2:
    parent_league2 = random.choice(population)

# Show parent fitness
print("\n--- Parent League 1 ---")
for i, team in enumerate(parent_league1.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(parent_league1):.2f}")

print("\n--- Parent League 2 ---")
for i, team in enumerate(parent_league2.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(parent_league2):.2f}")

# Apply position-based crossover
child1, child2 = preset_team_mix_crossover(parent_league1, parent_league2)

# Show child fitness
print("\n--- Child League 1 ---")
for i, team in enumerate(child1.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child1):.2f}")  # <-- this is fixed

print("\n--- Child League 2 ---")
for i, team in enumerate(child2.teams):
    print(f"Team {i+1} - Avg Skill: {team.get_average_skill():.2f} | Budget: {team.get_total_salary():.2f}")
print(f"Fitness: {calculate_fitness(child2):.2f}")


--- Parent League 1 ---
Team 1 - Avg Skill: 85.29 | Budget: 670.00
Team 2 - Avg Skill: 85.43 | Budget: 647.00
Team 3 - Avg Skill: 86.43 | Budget: 702.00
Team 4 - Avg Skill: 87.29 | Budget: 705.00
Team 5 - Avg Skill: 87.57 | Budget: 700.00
Fitness: 0.93

--- Parent League 2 ---
Team 1 - Avg Skill: 87.86 | Budget: 745.00
Team 2 - Avg Skill: 87.43 | Budget: 725.00
Team 3 - Avg Skill: 83.43 | Budget: 585.00
Team 4 - Avg Skill: 86.71 | Budget: 684.00
Team 5 - Avg Skill: 86.57 | Budget: 685.00
Fitness: 1.56


ValueError: Player Elijah Sanders is already in another team.

In [ ]:
# this is also working since i tried running multiple times and it returns as fitness 9999